In [2]:
import pandas as pd
from data_structures.Entity import Entity
import pickle as pkl

## Read datasets and extract for desired year

In [3]:
# Read datasets
df_title_principals = pd.read_csv('data/title.principals.tsv', sep='\t')
df_title_basics = pd.read_csv('data/title.basics.tsv', sep='\t')
df_name_basics = pd.read_csv('data/name.basics.tsv', sep='\t')

C:\Users\Joker\AppData\Local\Temp\ipykernel_6564\3015868806.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('data/title.basics.tsv', sep='\t')


In [22]:
df_title_basics.loc[df_title_basics["startYear"] == "\\N", "startYear"] = 0
df_title_basics.loc[df_title_basics["endYear"] == "\\N", "endYear"] = 0

df_title_basics["startYear"] = pd.to_numeric(df_title_basics["startYear"])
df_title_basics["endYear"] = pd.to_numeric(df_title_basics["endYear"])

In [42]:
year = 2013
print(len(df_title_basics))
print(len(df_title_basics[df_title_basics['startYear'] == year]))

df_title_basics_before2013 = df_title_basics[df_title_basics['startYear'] < 2013]
df_title_basics_within2013 = df_title_basics_before2013[df_title_basics_before2013['endYear'] >= 2013]
print(len(df_title_basics_before2013))
print(len(df_title_basics_within2013))

10225586
346603
5830980
5239


In [43]:
df_title_basics_only2013 = df_title_basics[df_title_basics['startYear'] == year]
df_title_basics_2013 = pd.concat([df_title_basics_only2013, df_title_basics_within2013])

with open("data/df_title_basics_2013.pkl", 'wb') as file:
    pkl.dump(df_title_basics_2013, file)

print(len(df_title_basics_2013)) # 351842

351842


In [45]:
df_title_basics_2013.head(10)
print(df_title_basics_2013[df_title_basics_2013["endYear"] != 0])

              tconst     titleType                     primaryTitle  \
971927    tt10007794      tvSeries              Varudhini Parinayam   
972201    tt10008268      tvSeries             Southern Fried Bride   
974052    tt10011672      tvSeries                   Motor Morphers   
975597    tt10014412      tvSeries                       Parasparam   
979907    tt10022096      tvSeries                       Trepadores   
...              ...           ...                              ...   
10208001   tt9878816      tvSeries                     Police Files   
10208649   tt9880204      tvSeries                      Royal Rasoi   
10209293   tt9881574  tvMiniSeries  La calle, el aula y la pantalla   
10219287   tt9903240      tvSeries                    The Leon Show   
10219774   tt9904298      tvSeries          Star Trek: Enterprise E   

                            originalTitle isAdult  startYear  endYear  \
971927                Varudhini Parinayam       0       2013     2016   
9

In [46]:
df_title_principals.head(10)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm1770680,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


## Make films: id->name dict

In [4]:
# Map all ids to films
films = {}

for index, row in df_title_basics.iterrows():
    title_id = str(row['tconst'])
    title_name = str(row['originalTitle'])
    films[title_id] = title_name

with open("data/films.pkl", 'wb') as file:
    pkl.dump(films, file)

## Make people: id->name dict

In [5]:
# Map all ids to people
people = {}

for index, row in df_name_basics.iterrows():
    name_id = str(row['nconst'])
    name = str(row['primaryName'])
    people[name_id] = name

with open("data/people.pkl", 'wb') as file:
    pkl.dump(people, file)

## Make films, people dict for given year

In [5]:
# Load saved datasets

with open("data/peopleids_principals.pkl", "rb") as file:
    peopleids_principals = pkl.load(file)

with open("data/films.pkl", "rb") as file:
    films = pkl.load(file)

with open("data/df_title_basics_2013.pkl", "rb") as file:
    df_title_basics_2013 = pkl.load(file)

In [6]:
print(type(peopleids_principals))
print(type(films))
print(type(df_title_basics_2013))

<class 'dict'>
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>


In [7]:
items_for_year = {"Films": [], "People": []}

# Get film names
film_originalTitle = list(set(df_title_basics_2013['originalTitle']))
items_for_year["Films"] = film_originalTitle

len(items_for_year["Films"])

# 4621927

219598

In [6]:
# Get people names
peopleids_principals = df_title_principals.groupby("tconst")["nconst"].agg(list).to_dict()
peopleids_principals

NameError: name 'df_title_principals' is not defined

In [19]:
# Dump dictionary
with open("data/peopleids_principals.pkl", 'wb') as file:
    pkl.dump(peopleids_principals, file)

In [4]:
peopleids_principals

{'tt0000001': ['nm1588970', 'nm0005690', 'nm0374658'],
 'tt0000002': ['nm0721526', 'nm1335271'],
 'tt0000003': ['nm0721526', 'nm1770680', 'nm1335271', 'nm5442200'],
 'tt0000004': ['nm0721526', 'nm1335271'],
 'tt0000005': ['nm0443482', 'nm0653042', 'nm0005690', 'nm0249379'],
 'tt0000006': ['nm0005690'],
 'tt0000007': ['nm0179163',
  'nm0183947',
  'nm0005690',
  'nm0374658',
  'nm0249379'],
 'tt0000008': ['nm0653028', 'nm0005690', 'nm0374658'],
 'tt0000009': ['nm0063086', 'nm0183823', 'nm1309758', 'nm0085156'],
 'tt0000010': ['nm0525910'],
 'tt0000011': ['nm3692297', 'nm0804434'],
 'tt0000012': ['nm2880396',
  'nm9735580',
  'nm0525900',
  'nm9735581',
  'nm0525908',
  'nm0525910',
  'nm9735579',
  'nm9653419'],
 'tt0000013': ['nm0525908', 'nm1715062', 'nm0525910'],
 'tt0000014': ['nm0166380', 'nm0244989', 'nm0525910'],
 'tt0000015': ['nm0721526'],
 'tt0000016': ['nm0525900', 'nm9735581', 'nm0525910'],
 'tt0000017': ['nm3691272', 'nm3692829', 'nm1587194', 'nm0804434'],
 'tt0000018': ['n

In [10]:
films

{'tt0000001': 'Carmencita',
 'tt0000002': 'Le clown et ses chiens',
 'tt0000003': 'Pauvre Pierrot',
 'tt0000004': 'Un bon bock',
 'tt0000005': 'Blacksmith Scene',
 'tt0000006': 'Chinese Opium Den',
 'tt0000007': 'Corbett and Courtney Before the Kinetograph',
 'tt0000008': 'Edison Kinetoscopic Record of a Sneeze',
 'tt0000009': 'Miss Jerry',
 'tt0000010': "La sortie de l'usine Lumière à Lyon",
 'tt0000011': 'Akrobatisches Potpourri',
 'tt0000012': "L'arrivée d'un train à La Ciotat",
 'tt0000013': 'Le débarquement du congrès de photographie à Lyon',
 'tt0000014': "L'arroseur arrosé",
 'tt0000015': "Autour d'une cabine",
 'tt0000016': 'Barque sortant du port',
 'tt0000017': 'Italienischer Bauerntanz',
 'tt0000018': 'Das boxende Känguruh',
 'tt0000019': 'The Clown Barber',
 'tt0000020': 'The Derby 1895',
 'tt0000022': 'Les forgerons',
 'tt0000023': 'Baignade en mer',
 'tt0000024': 'Opening of the Kiel Canal',
 'tt0000025': 'The Oxford and Cambridge University Boat Race',
 'tt0000026': "Par

In [ ]:
# Get 'year' people
people_for_year = []
print(len(peopleids_principals.items()))

for t, p in peopleids_principals.items():
    try:
        t_name = films[t]
        if t_name in items_for_year["Films"]:
            people_for_year.extend(p)
    except:
        continue

pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass
pass2
pass3
pass

In [12]:
with open("data/people_for_year.pkl", 'wb') as file:
    pkl.dump(people_for_year, file)

people_for_year

[]

## Obsolete

In [6]:
count = 0
for index, row in df_title_basics_certainyear.iterrows(): #
    # Get attributes, mainly for the year
    title_name = films[str(row['tconst'])]
    title_startyear = str(row['startYear'])
    title_endyear = str(row['endYear'])

    # Distinguish TV series from films
    if title_endyear != "\\N": # TV series
        years_to_add = [str(i) for i in range(int(title_startyear), int(title_endyear)+1)]
    elif title_startyear == "\\N": # unknown
        continue
    else: # film
        years_to_add = [title_startyear]

    # Get people in film/series
    df_title_people_ids = df_title_principals[df_title_principals['tconst'] == title_id]
    people_names = []

    for index_people, row_people in df_title_people_ids.iterrows():
        person_name_lst = df_name_basics[df_name_basics['nconst'] == row_people['nconst']]['primaryName'].tolist()
        people_names.append(Entity(person_name_lst[0]))
    
    #print("PEOPLE NAMES: ", people_names)

    # for each year in years_to_add, add those people and the film name
    for year in years_to_add:
        film_entity = Entity(title_name)
        items_for_year["Films"].append(film_entity)
        items_for_year["People"].extend(people_names)

        #print(year, years[year])
    #print(count)
    count += 1

with open("data/"+str(year)+".pkl", 'wb') as file:
    pkl.dump(items_for_year, file)
